In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/dy/909n7xmj5_70npk7nknw102m0000gn/T/ipykernel_16326/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [3]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("/Users/diegoalonso/Documents/Ironhack/Labs/Week_4/Day_2/lab-natural-language-processing/data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [4]:
from sklearn.model_selection import train_test_split
# We´ll use data_train and data_val as it will be used on the 'Extra Features' section with this naming
data_train, data_val = train_test_split(data, test_size=0.2, random_state=42, stratify=data['label'])

## Data Preprocessing

In [5]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [6]:
import re #We will use regular expressions to clean the data

# Function to remove inline JavaScript/CSS, HTML comments, and remaining tags
def remove_inline_code(text):
    # First we remove inline JavaScript
    text = re.sub(r'<script>.*?</script>', '', text, flags=re.DOTALL)
    # Then we must remove inline CSS
    text = re.sub(r'<style>.*?</style>', '', text, flags=re.DOTALL)
    # Then we remove HTML comments before removing tags
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    # Last we remove remaining tags
    text = re.sub(r'<.*?>', '', text)
    return text

# Apply the function to the data 
data_train['preprocessed_text'] = data_train['text'].apply(remove_inline_code)
data_val['preprocessed_text'] = data_val['text'].apply(remove_inline_code)

# We check if the data has been cleaned correctly
data_train.head(10)

,text,label,preprocessed_text
442,Dear=2C Good day hope fine=2Cdear am writting ...,1,Dear=2C Good day hope fine=2Cdear am writting ...
962,FROM MR HENRY KABORETHE CHIEF AUDITOR INCHARGE...,1,FROM MR HENRY KABORETHE CHIEF AUDITOR INCHARGE...
971,Will do.,0,Will do.
190,FROM THE DESK OF DR.ADAMU ISMALERAUDITING AND...,1,FROM THE DESK OF DR.ADAMU ISMALERAUDITING AND...
551,"Dear Friend, My name is LOI C.ESTRADA,The wife...",1,"Dear Friend, My name is LOI C.ESTRADA,The wife..."
495,FROM =3AMR=2ECOLLINCE ADDOATTN =3AI NEED YOUR ...,1,FROM =3AMR=2ECOLLINCE ADDOATTN =3AI NEED YOUR ...
845,Pls send me call sheet for Equadoran--i don't ...,0,Pls send me call sheet for Equadoran--i don't ...
821,Yes I will arrange.,0,Yes I will arrange.
409,FYI Ã¢ÂÂ I will limit Haiti emails to major ...,0,FYI Ã¢ÂÂ I will limit Haiti emails to major ...
794,Mr=2E Zuhair Idris Jordan Kuwait BankCredit De...,1,Mr=2E Zuhair Idris Jordan Kuwait BankCredit De...


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [7]:
# Function to preprocess text
def remove_and_convert(text):
    # Remove all special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove numbers
    text = re.sub(r'\d', '', text)
    
    # Remove all single characters
    text = re.sub(r'\b\w\b', '', text)
    
    # Remove single characters from the start
    text = re.sub(r'^\w\s', '', text)
    
    # Substitute multiple spaces with single space
    text = re.sub(r'\s+', ' ', text)
    
    # Remove prefixed 'b'
    text = re.sub(r'^b', '', text)
    
    # Convert to Lowercase
    text = text.lower()
    
    return text

# Apply the function to the data 
data_train['preprocessed_text'] = data_train['preprocessed_text'].apply(remove_and_convert)
data_val['preprocessed_text'] = data_val['text'].apply(remove_and_convert)

# We check if the data has been cleaned correctly
data_train.head(10)

,text,label,preprocessed_text
442,Dear=2C Good day hope fine=2Cdear am writting ...,1,dearc good day hope finecdear am writting this...
962,FROM MR HENRY KABORETHE CHIEF AUDITOR INCHARGE...,1,from mr henry kaborethe chief auditor incharge...
971,Will do.,0,will do
190,FROM THE DESK OF DR.ADAMU ISMALERAUDITING AND...,1,from the desk of dradamu ismalerauditing and a...
551,"Dear Friend, My name is LOI C.ESTRADA,The wife...",1,dear friend my name is loi cestradathe wife of...
495,FROM =3AMR=2ECOLLINCE ADDOATTN =3AI NEED YOUR ...,1,from amrecollince addoattn ai need your urgent...
845,Pls send me call sheet for Equadoran--i don't ...,0,pls send me call sheet for equadorani dont hav...
821,Yes I will arrange.,0,yes will arrange
409,FYI Ã¢ÂÂ I will limit Haiti emails to major ...,0,fyi will limit haiti emails to major fyi infor...
794,Mr=2E Zuhair Idris Jordan Kuwait BankCredit De...,1,mre zuhair idris jordan kuwait bankcredit depa...


## Now let's work on removing stopwords
Remove the stopwords.

In [8]:
# Function to remove stopwords
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

# Apply the function to the data
data_train['preprocessed_text'] = data_train['preprocessed_text'].apply(remove_stopwords)
data_val['preprocessed_text'] = data_val['preprocessed_text'].apply(remove_stopwords)  

data_train.head(10)

,text,label,preprocessed_text
442,Dear=2C Good day hope fine=2Cdear am writting ...,1,dearc good day hope finecdear writting mail du...
962,FROM MR HENRY KABORETHE CHIEF AUDITOR INCHARGE...,1,mr henry kaborethe chief auditor inchargeforei...
971,Will do.,0,
190,FROM THE DESK OF DR.ADAMU ISMALERAUDITING AND...,1,desk dradamu ismalerauditing accounting manage...
551,"Dear Friend, My name is LOI C.ESTRADA,The wife...",1,dear friend name loi cestradathe wife mr josep...
495,FROM =3AMR=2ECOLLINCE ADDOATTN =3AI NEED YOUR ...,1,amrecollince addoattn ai need urgent assistanc...
845,Pls send me call sheet for Equadoran--i don't ...,0,pls send call sheet equadorani dont clue
821,Yes I will arrange.,0,yes arrange
409,FYI Ã¢ÂÂ I will limit Haiti emails to major ...,0,fyi limit haiti emails major fyi information
794,Mr=2E Zuhair Idris Jordan Kuwait BankCredit De...,1,mre zuhair idris jordan kuwait bankcredit depa...


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [9]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Function to lemmatize and tokenize
def tokenize_and_lemmatize(text):
    # Check if the input is a list
    if isinstance(text, list):
        # If it's a list, join the words back into a string
        text = ' '.join(text)
    
    # Tokenize the text into individual words
    tokens = word_tokenize(text)
    
    # Create a lemmatizer object
    lemmatizer = WordNetLemmatizer()
    
    # Lemmatize each token and return the result
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Join the lemmatized tokens back into a string
    return ' '.join(lemmatized_tokens)

# Apply the function to the data
data_train['preprocessed_text'] = data_train['preprocessed_text'].apply(tokenize_and_lemmatize)
data_val['preprocessed_text'] = data_val['preprocessed_text'].apply(tokenize_and_lemmatize)

data_train.head(10)

,text,label,preprocessed_text
442,Dear=2C Good day hope fine=2Cdear am writting ...,1,dearc good day hope finecdear writting mail du...
962,FROM MR HENRY KABORETHE CHIEF AUDITOR INCHARGE...,1,mr henry kaborethe chief auditor inchargeforei...
971,Will do.,0,
190,FROM THE DESK OF DR.ADAMU ISMALERAUDITING AND...,1,desk dradamu ismalerauditing accounting manage...
551,"Dear Friend, My name is LOI C.ESTRADA,The wife...",1,dear friend name loi cestradathe wife mr josep...
495,FROM =3AMR=2ECOLLINCE ADDOATTN =3AI NEED YOUR ...,1,amrecollince addoattn ai need urgent assistanc...
845,Pls send me call sheet for Equadoran--i don't ...,0,pls send call sheet equadorani dont clue
821,Yes I will arrange.,0,yes arrange
409,FYI Ã¢ÂÂ I will limit Haiti emails to major ...,0,fyi limit haiti email major fyi information
794,Mr=2E Zuhair Idris Jordan Kuwait BankCredit De...,1,mre zuhair idris jordan kuwait bankcredit depa...


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [10]:
from collections import Counter

# Split the data into ham and spam messages
ham_messages = data_train[data_train['label'] == 0]['text']
spam_messages = data_train[data_train['label'] == 1]['text']

# Count the frequency of each word in the ham messages
ham_words = Counter(' '.join(ham_messages).split())

# Count the frequency of each word in the spam messages
spam_words = Counter(' '.join(spam_messages).split())

# Get the top 10 words in the ham messages
top_ham_words = ham_words.most_common(10)

# Get the top 10 words in the spam messages
top_spam_words = spam_words.most_common(10)

# Print the results
print("Top 10 words in ham messages:")
for word, count in top_ham_words:
    print(f"{word}: {count}")

print("\nTop 10 words in spam messages:")
for word, count in top_spam_words:
    print(f"{word}: {count}")

Top 10 words in ham messages:
the: 1433
to: 912
and: 690
of: 682
a: 532
in: 483
that: 338
for: 318
is: 302
on: 253

Top 10 words in spam messages:
the: 4654
to: 3741
of: 3373
and: 2618
in: 2162
I: 1990
you: 1838
this: 1588
a: 1556
your: 1352


## Extra features

In [11]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

,text,label,preprocessed_text,money_mark,suspicious_words,text_len
442,Dear=2C Good day hope fine=2Cdear am writting ...,1,dearc good day hope finecdear writting mail du...,1,1,1024
962,FROM MR HENRY KABORETHE CHIEF AUDITOR INCHARGE...,1,mr henry kaborethe chief auditor inchargeforei...,1,1,1954
971,Will do.,0,,1,0,0
190,FROM THE DESK OF DR.ADAMU ISMALERAUDITING AND...,1,desk dradamu ismalerauditing accounting manage...,1,1,390
551,"Dear Friend, My name is LOI C.ESTRADA,The wife...",1,dear friend name loi cestradathe wife mr josep...,1,1,1507


## How would work the Bag of Words with Count Vectorizer concept?

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
# To make Bag of Words work with Coubt Vectorizer, we will have to combine the messages so that we can convert it to a vector so that we can apply the it to a classifier.
# We will join every message in a link them with blank spaces. ".iloc" is Purely integer-location based indexing for selection by position. 

messages = []
for row in range(0,len(data_train.index)):
    messages.append(str(data_train['preprocessed_text'].iloc[row]))

print (messages [0])

# implement BAG OF WORDS
countvector=CountVectorizer(ngram_range=(1,2))
traindataset=countvector.fit_transform(messages)

dearc good day hope finecdear writting mail due respect heartful tear since known met previously asking assistanceci glad render assistance situation nowe make proposal well known given opportunitye would like use opportunity introduce youe miss johana johnpaul year old girl liberia cthe daughter late godwin johnpaul deputy minister national security leadership president charles taylor liberia exile many innocent soul killede father killed government charlestaylorcheaccuse father coup attempt month mother cynthia also killede main reason contacting seek assistance area future investment also want hand huge amount money youe money ten millon five hundred thousand u dollar deposited year ago father made sole beneficiaryfnext kin moneye asking stand behalf make claim bankcam girl young age cannt handle transactionci want stand foregin partner oversea also help investment money well send telephone number picture okeeee original copy document concering money herecso please expecting hear so

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Create a CountVectorizer object for TF-IDF
tfidfvector = TfidfVectorizer()

# Vectorize the messages dataset
vectorized_messages = tfidfvector.fit_transform(messages)

# Print the shape of the vectorized messages
print(vectorized_messages.shape)


(800, 17179)


## And the Train a Classifier?

In [14]:
from sklearn.naive_bayes import MultinomialNB

# Fit the Naive Bayes model
naive=MultinomialNB()
naive.fit(traindataset,data_train['label'])

MultinomialNB()

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [23]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data_train['preprocessed_text'], data_train['label'], test_size=0.2, random_state=42)

# Define the classifier
clf = naive

# TF-IDF
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

clf.fit(X_train_vectorized, y_train)
y_pred = clf.predict(X_test_vectorized)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Feature Extraction Method: TF-IDF")
matrix=confusion_matrix(y_test, y_pred)
print(matrix) # print confusion matrix
score=accuracy_score(y_test, y_pred)
print(f'The accuracy score is: {score}') # print(score)
report=classification_report(y_test, y_pred)
print(report) # print report

# CountVectorizer
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

clf.fit(X_train_vectorized, y_train)
y_pred = clf.predict(X_test_vectorized)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Feature Extraction Method: CountVectorizer")
matrix=confusion_matrix(y_test, y_pred)
print(matrix)
score=accuracy_score(y_test, y_pred)
print(f'The accuracy score is: {score}')
report=classification_report(y_test, y_pred)
print(report)

# TF-IDF (ngram_range=(1,2))
vectorizer = TfidfVectorizer(ngram_range=(1,2))
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

clf.fit(X_train_vectorized, y_train)
y_pred = clf.predict(X_test_vectorized)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Feature Extraction Method: TF-IDF (ngram_range=(1,2))")
matrix=confusion_matrix(y_test, y_pred)
print(matrix)
score=accuracy_score(y_test, y_pred)
print(f'The accuracy score is: {score}')
report=classification_report(y_test, y_pred)
print(report)

# CountVectorizer (ngram_range=(1,2))
vectorizer = CountVectorizer(ngram_range=(1,2))
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

clf.fit(X_train_vectorized, y_train)
y_pred = clf.predict(X_test_vectorized)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Feature Extraction Method: CountVectorizer (ngram_range=(1,2))")
matrix=confusion_matrix(y_test, y_pred)
print(matrix)
score=accuracy_score(y_test, y_pred)
print(f'The accuracy score is: {score}')
report=classification_report(y_test, y_pred)
print(report)

# TF-IDF (ngram_range=(2,2))
vectorizer = TfidfVectorizer(ngram_range=(2,2))
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

clf.fit(X_train_vectorized, y_train)
y_pred = clf.predict(X_test_vectorized)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Feature Extraction Method: TF-IDF (ngram_range=(2,2))")
matrix=confusion_matrix(y_test, y_pred)
print(matrix)
score=accuracy_score(y_test, y_pred)
print(f'The accuracy score is: {score}')
report=classification_report(y_test, y_pred)
print(report)

# CountVectorizer (ngram_range=(2,2))
vectorizer = CountVectorizer(ngram_range=(2,2))
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

clf.fit(X_train_vectorized, y_train)
y_pred = clf.predict(X_test_vectorized)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Feature Extraction Method: CountVectorizer (ngram_range=(2,2))")
matrix=confusion_matrix(y_test, y_pred)
print(matrix)
score=accuracy_score(y_test, y_pred)
print(f'The accuracy score is: {score}')
report=classification_report(y_test, y_pred)
print(report)

Feature Extraction Method: TF-IDF
[[74 15]
 [ 6 65]]
The accuracy score is: 0.86875
              precision    recall  f1-score   support

           0       0.93      0.83      0.88        89
           1       0.81      0.92      0.86        71

    accuracy                           0.87       160
   macro avg       0.87      0.87      0.87       160
weighted avg       0.88      0.87      0.87       160

Feature Extraction Method: CountVectorizer
[[79 10]
 [ 6 65]]
The accuracy score is: 0.9
              precision    recall  f1-score   support

           0       0.93      0.89      0.91        89
           1       0.87      0.92      0.89        71

    accuracy                           0.90       160
   macro avg       0.90      0.90      0.90       160
weighted avg       0.90      0.90      0.90       160

Feature Extraction Method: TF-IDF (ngram_range=(1,2))
[[74 15]
 [ 6 65]]
The accuracy score is: 0.86875
              precision    recall  f1-score   support

           0  

The two best classifiers are TF-IDF and CountVectorizer with ngram_range=(2,2)